# REDNet Autoencoder

In [47]:
import os
WORKDIR = os.path.realpath("D:/OneDrive/Code/rednet_tensorflow/")
os.chdir(WORKDIR)
print("Current Dir: ", os.getcwd())

Current Dir:  D:\OneDrive\Code\rednet_tensorflow


In [48]:
from __future__ import print_function, absolute_import, with_statement
from IPython import display as ipythondisplay
import tensorflow as tf
tf.enable_eager_execution()
print(tf.__version__)

import h5py
#save_keras_model, plot_progress, plot_loss_acc, myplot, myplot_history
from dl_utils import *

# REDNet
from rednet import *
from DatasetLoader import HDF5DatasetLoader

1.14.0


In [49]:
data_local = True

db_share = "https://www.dropbox.com/s/l5iqduhe0gwxumq/train_face.h5?dl=1"
dataset_name = "train_face.h5"
dataset_dir = os.path.join(WORKDIR, "datasets")
dataset_path = os.path.join(dataset_dir, dataset_name)

# Get the training data: both images from CelebA and ImageNet
if h5py.is_hdf5(dataset_path) is True:
  print("Local Dataset Path: " + dataset_path)
else:
  path_to_dataset = tf.keras.utils.get_file(dataset_name, db_share)#,cache_dir=DATADIR)
  print("Dropbox Dataset Path: " + path_to_dataset)

Local Dataset Path: D:\OneDrive\Code\rednet_tensorflow\datasets\train_face.h5


In [50]:
# Instantiate a TrainingDatasetLoader using the downloaded dataset
loader = HDF5DatasetLoader(dataset_path)
print(loader.image_dims)
print(loader.train_inds)

Opening D:\OneDrive\Code\rednet_tensorflow\datasets\train_face.h5
Loading data into memory...
(109914, 64, 64, 3)
[     0      1      2 ... 109911 109912 109913]


In [ ]:
rednet = REDNet(loss_fn='binary')


In [3]:
opt = tf.keras.optimizers.Adam(lr=0.0001)
rednet.compile(loss='mean_squared_error', 
               optimizer=opt, 
               metrics=['accuracy', 'mae'])

In [10]:
%cd D:/OneDrive/Code/rednet_tensorflow

D:\OneDrive\Code\rednet_tensorflow
